In [155]:
import keras
from keras import layers
import numpy as np
from matplotlib import pyplot

In [156]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [157]:
generator_input = keras.Input(shape=(latent_dim,))

In [158]:
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)

In [159]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_82 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_83 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 32, 32, 256)       1048832   
__________

In [160]:
discriminator_input = layers.Input(shape=(height, width, channels))

In [161]:
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

In [162]:
discriminator = keras.models.Model(discriminator_input, x)

In [163]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_87 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_88 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_89 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [164]:
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [165]:
discriminator.trainable = False

In [166]:
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [167]:
import os
from keras.preprocessing import image

In [168]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [169]:
x_train = x_train[y_train.flatten() == 6] # this gets only frogs (class 6)
print(x_train.shape)
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.0 # normalize the data

iterations = 10000
batch_size = 20
save_dir = 'D:\\' # where images should be saved

start = 0

(5000, 32, 32, 3)


In [170]:
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
                                                  latent_dim))
    
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = x_train[start: stop]

    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)),
                            np.zeros((batch_size, 1))])
    labels += 0.5 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_thing' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_thing' + str(step) + '.png'))

discriminator loss: 0.69491327
adversarial loss: 0.71845776
discriminator loss: 0.68636453
adversarial loss: 2.4429862
discriminator loss: 0.5947443
adversarial loss: 1.3014187
discriminator loss: 0.5283569
adversarial loss: 1.5581124
discriminator loss: 0.5656101
adversarial loss: 1.327819
discriminator loss: 0.55487084
adversarial loss: 1.4244235
discriminator loss: 0.577783
adversarial loss: 1.3274149
discriminator loss: 0.60384715
adversarial loss: 1.2755077
discriminator loss: 0.5517017
adversarial loss: 1.4209207
discriminator loss: 0.651693
adversarial loss: 1.3093557
discriminator loss: 0.52268875
adversarial loss: 1.5373191
discriminator loss: 0.56284124
adversarial loss: 1.1787459
discriminator loss: 0.64179146
adversarial loss: 1.3776186
discriminator loss: 0.5879229
adversarial loss: 1.501638
discriminator loss: 0.52713233
adversarial loss: 1.5165293
discriminator loss: 0.57807666
adversarial loss: 1.3248559
discriminator loss: 0.56242114
adversarial loss: 1.3757375
discrim